# Wealth transfer

This notebook presents a tutorial for beginners on how to create a simple agent-based model with the [agentpy](https://agentpy.readthedocs.io) package.
It demonstrates how to create a basic model with a custom agent type, run a simulation, record data, and visualize results.

In [ ]:
# Model design
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns

## About the model

The model explores the distribution of wealth under a trading population of agents.
Each agent starts with one unit of wealth.
During each time-step, each agents with positive wealth
randomly selects a trading partner and gives them one unit of their wealth.
We will see that this random interaction will create an inequality of wealth that
follows a [Boltzmann distribution](http://www.phys.ufl.edu/~meisel/Boltzmann.pdf).
The original version of this model been written in [MESA](https://mesa.readthedocs.io/)
and can be found [here](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html).

## Model definition

We start by defining a new type of `Agent` with the following methods:

- `setup()` is called automatically when a new agent is created and initializes a variable `wealth`.
- `wealth_transfer()` describes the agent's behavior at every time-step and will be called by the model.

In [ ]:
class CleaningAgent1(ap.Agent):

     #Rodo

    def setup(self):
        self.position = [0, 0]
        self.utility = 0
        self.target_area = None

    def see(self):
        """ observa el ambiente y revisa el area con mas densidad"""
        self.environment = self.model.environment
        max_density = 0
        self.target_area = None

        # evalua la densidad que hay en espacio de 3x3
        for i in range(len(self.environment)):
            for j in range(len(self.environment[0])):
                density = 0
                for di in range(-1, 2):
                    for dj in range(-1, 2):
                        ni, nj = i + di, j + dj
                        if 0 <= ni < len(self.environment) and 0 <= nj < len(self.environment[0]):
                            density += self.environment[ni][nj]
                if density > max_density:
                    max_density = density
                    self.target_area = [i, j]

    def move_toward_target(self):
        """ se mueve hacia el area donde hubo mas densidad en el "see"""
        if self.target_area:
            if self.target_area[0] < self.position[0]:
                self.position[0] -= 1
            elif self.target_area[0] > self.position[0]:
                self.position[0] += 1
            elif self.target_area[1] < self.position[1]:
                self.position[1] -= 1
            elif self.target_area[1] > self.position[1]:
                self.position[1] += 1

    def clean(self):
        """ limpia la celda actual si esta sucia"""
        if self.environment[self.position[0]][self.position[1]]:
            self.environment[self.position[0]][self.position[1]] = False
            self.utility += 1

    def step(self):

        self.see()
        self.clean()
        self.move_toward_target()

class CleaningAgent2(ap.Agent):

    def setup(self):
        self.utility = 0

    """ An agent with wealth "- Sofia

    def setup(self):

        self.wealth = 1
        self.utility = 1

    def wealth_transfer(self):
      partner = (self.model.agents1 + self.model.agents2 + self.model.agents4).random()
      if partner.wealth < self.wealth:
            amount = min(2, self.wealth)
            self.wealth -= amount
            partner.wealth += amount
            self.utility += amount"""

class CleaningAgent3(ap.Agent):

    """ A cleaning agent  - Mariana"""

    def setup(self):
        self.position = [0, 0]
        self.utility = 0

    def see(self):
        self.environment = self.model.environment

    def next(self):
        self.goingto = [0, 0]
        if not self.goingto:
            m = 12
            for i in range(len(self.environment)):
                for j in range(len(self.environment[0])):
                    if self.environment[i][j] and (self.position[0] - i + self.position[1] - j) < m:
                        m = self.position[0] - i + self.position[1] - j
                        self.goingto = [i, j]

    def action(self):
        if self.environment[self.position[0]][self.position[1]]:
            self.model.environment[self.position[0]][self.position[1]] = False
            self.utility += 1
        else:
            if self.goingto[0] < self.position[0]:
                self.position[0] -= 1
            elif self.goingto[0] > self.position[0]:
                self.position[0] += 1
            elif self.goingto[1] < self.position[1]:
                self.position[1] -= 1
            elif self.goingto[1] > self.position[1]:
                self.position[1] += 1

    def step(self):
        self.see()
        self.next()
        self.action()

class CleaningAgent4(ap.Agent):

    def setup(self):
        self.utility = 0

    """ An agent with wealth - Roger

    def setup(self):

        self.wealth = np.random.randint(1, 101)
        self.utility = 1

    def wealth_transfer(self):

        if self.wealth > 0:

            partner = (self.model.agents1 + self.model.agents2 + self.model.agents4).random()
            if partner.wealth <= self.wealth:
                partner.wealth += 1
                self.wealth -= 1"""

Finally, we define our [`Model`](https://agentpy.readthedocs.io/en/stable/reference_models.html) with the following methods:

- `setup` defines how many agents should be created at the beginning of the simulation.
- `step` calls all agents during each time-step to perform their `wealth_transfer` method.
- `update` calculates and record the current Gini coefficient after each time-step.
- `end`, which is called at the end of the simulation, we record the wealth of each agent.

In [ ]:
class WealthModel(ap.Model):

    """ A simple model of random wealth transfers """

    def setup(self):

        m = 3
        n = 4
        k = 2

        self.environment = [[False] * m] * n

        for i in range(k):
            x = np.random.randint(n)
            y = np.random.randint(m)
            while self.environment[x][y]:
                x = np.random.randint(n)
                y = np.random.randint(m)
            self.environment[x][y] = True

        self.agents1 = ap.AgentList(self, self.p.agents, CleaningAgent1)
        self.agents2 = ap.AgentList(self, self.p.agents, CleaningAgent2)
        self.agents3 = ap.AgentList(self, self.p.agents, CleaningAgent3)
        self.agents4 = ap.AgentList(self, self.p.agents, CleaningAgent4)

    def step(self):

        self.agents3.step()

    def update(self):

        self.record('Utility (Agent 1)', list(self.agents1.utility))
        self.record('Utility (Agent 2)', list(self.agents2.utility))
        self.record('Utility (Agent 3)', list(self.agents3.utility))
        self.record('Utility (Agent 4)', list(self.agents4.utility))

    def end(self):

        """self.agents1.record('wealth')
        self.agents2.record('wealth')
        self.agents3.record('wealth')
        self.agents4.record('wealth')

        self.record("Task predicate 1", all(list(self.agents1.wealth)))
        self.record("Task predicate 2", all(list(self.agents2.wealth)))
        self.record("Task predicate 3", all(list(self.agents3.wealth)))
        self.record("Task predicate 4", all(list(self.agents4.wealth)))"""

## Simulation run

In [ ]:
parameters = {
    'agents': 50,
    'steps': 100,
    'seed': 42,
}

In [ ]:
model = WealthModel(parameters)
results = model.run()

Completed: 100 steps
Run time: 0:00:00.018877
Simulation finished


## Output analysis

The simulation returns a [`DataDict`](https://agentpy.readthedocs.io/en/stable/reference_output.html) with our recorded variables.

In [ ]:
results

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 3 keys
'variables': 
    'WealthModel': DataFrame with 4 variables and 101 rows
'reporters': DataFrame with 1 variable and 1 row
}

The output's `info` provides general information about the simulation.

In [ ]:
results.info

{'model_type': 'WealthModel',
 'time_stamp': '2025-01-13 22:37:17',
 'agentpy_version': '0.1.5',
 'python_version': '3.12.',
 'experiment': False,
 'completed': True,
 'created_objects': 200,
 'completed_steps': 100,
 'run_time': '0:00:00.018877'}

To explore the evolution of inequality,
we look at the recorded [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) of the model's variables.

In [ ]:
results.variables.WealthModel.head()

,Utility (Agent 1),Utility (Agent 2),Utility (Agent 3),Utility (Agent 4)
t,,,,
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


To visualize this data,
we can use [`DataFrame.plot`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html).

To look at the distribution at the end of the simulation,
we visualize the recorded agent variables with [seaborn](https://seaborn.pydata.org/).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

"""fig, axs = plt.subplots(nrows=4)
df = pd.DataFrame(list(data["Utility (Agent 1)"]))
sns.stripplot(data=df, ax=axs[0]);
df = pd.DataFrame([list(data["Utility (Agent 2)"])[99]])
sns.stripplot(data=df, ax=axs[1]);
df = pd.DataFrame([list(data["Utility (Agent 3)"])[99]])
sns.stripplot(data=df, ax=axs[2])
df = pd.DataFrame(list(data["Utility (Agent 4)"]))
sns.stripplot(data=df, ax=axs[3]);"""

'fig, axs = plt.subplots(nrows=4)\ndf = pd.DataFrame(list(data["Utility (Agent 1)"]))\nsns.stripplot(data=df, ax=axs[0]);\ndf = pd.DataFrame([list(data["Utility (Agent 2)"])[99]])\nsns.stripplot(data=df, ax=axs[1]);\ndf = pd.DataFrame([list(data["Utility (Agent 3)"])[99]])\nsns.stripplot(data=df, ax=axs[2])\ndf = pd.DataFrame(list(data["Utility (Agent 4)"]))\nsns.stripplot(data=df, ax=axs[3]);'